<a href="https://colab.research.google.com/github/dkraynev/Tasks/blob/main/384_Page_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача: Рекомендация страниц на основе лайков друзей

## Условие задачи

У нас есть две таблицы: `Friendship` и `Likes`.

### Таблица Friendship:
| Column Name   | Type    |
|---------------|---------|
| user1_id      | int     |
| user2_id      | int     |

- `(user1_id, user2_id)` является первичным ключом.
- Каждая строка этой таблицы указывает, что существует дружеская связь между `user1_id` и `user2_id`.

### Таблица Likes:
| Column Name | Type    |
|-------------|---------|
| user_id     | int     |
| page_id     | int     |

- `(user_id, page_id)` является первичным ключом.
- Каждая строка этой таблицы указывает, что `user_id` понравилась страница `page_id`.

### Задача:
Необходимо написать решение для рекомендаций страниц пользователю с `user_id = 1` на основе страниц, которые лайкнули его друзья. При этом не следует рекомендовать страницы, которые пользователь уже лайкнул.

**Формат результата**: таблица с рекомендуемыми страницами в любом порядке без дубликатов.

### Пример 1:

**Входные данные:**

Таблица `Friendship`:

| user1_id | user2_id |
|----------|----------|
| 1        | 2        |
| 1        | 3        |
| 1        | 4        |
| 2        | 3        |
| 2        | 4        |
| 2        | 5        |
| 6        | 1        |

Таблица `Likes`:

| user_id | page_id |
|---------|---------|
| 1       | 88      |
| 2       | 23      |
| 3       | 24      |
| 4       | 56      |
| 5       | 11      |
| 6       | 33      |
| 2       | 77      |
| 3       | 77      |
| 6       | 88      |

**Выходные данные:**

| recommended_page |
|------------------|
| 23               |
| 24               |
| 56               |
| 33               |
| 77               |

**Объяснение:**
- Пользователь с `user_id = 1` является другом пользователей 2, 3, 4 и 6.
- Рекомендуемые страницы: 23 от пользователя 2, 24 от пользователя 3, 56 от пользователя 4, и 33 от пользователя 6.
- Страница 77 была предложена как от пользователя 2, так и от пользователя 3.
- Страница 88 не была предложена, так как пользователь с `user_id = 1` уже лайкнул её.


In [ ]:
import pandas as pd

# Данные о дружбе
friendship_data = {
    'user1_id': [1, 1, 1, 2, 2, 2, 6],
    'user2_id': [2, 3, 4, 3, 4, 5, 1]
}

# Преобразуем данные о дружбе в DataFrame
friendship_df = pd.DataFrame(friendship_data)

# Данные о лайках
likes_data = {
    'user_id': [1, 2, 3, 4, 5, 6, 2, 3, 6],
    'page_id': [88, 23, 24, 56, 11, 33, 77, 77, 88]
}

# Преобразуем данные о лайках в DataFrame
likes_df = pd.DataFrame(likes_data)

# Выводим данные для наглядности
print("Данные о дружбе:")
print(friendship_df)
print("\nДанные о лайках:")
print(likes_df)


Данные о дружбе:
   user1_id  user2_id
0         1         2
1         1         3
2         1         4
3         2         3
4         2         4
5         2         5
6         6         1

Данные о лайках:
   user_id  page_id
0        1       88
1        2       23
2        3       24
3        4       56
4        5       11
5        6       33
6        2       77
7        3       77
8        6       88


In [ ]:
# Ищем друзей пользователя с user_id = 1
friends_of_user_1 = friendship_df[
    (friendship_df['user1_id'] == 1) | (friendship_df['user2_id'] == 1)
]

# Преобразуем результаты в один список user_id друзей
# Если user1_id = 1, то друг находится в user2_id, и наоборот
friends_of_user_1['friend_id'] = friends_of_user_1.apply(
    lambda row: row['user2_id'] if row['user1_id'] == 1 else row['user1_id'], axis=1
)

# Извлекаем только уникальные идентификаторы друзей
friends_list = friends_of_user_1['friend_id'].unique()

# Выводим список друзей
print("\nСписок друзей пользователя 1:")
print(friends_list)

In [ ]:
# Ищем все страницы, которые лайкнули друзья из friends_list
friends_likes = likes_df[likes_df['user_id'].isin(friends_list)]

# Выводим страницы, которые лайкнули друзья
print("\nСтраницы, которые лайкнули друзья пользователя 1:")
print(friends_likes)


In [ ]:
# Страницы, которые уже лайкнул пользователь 1
user_1_likes = likes_df[likes_df['user_id'] == 1]['page_id'].unique()

# Исключаем страницы, которые уже лайкнул пользователь 1
recommended_pages = friends_likes[~friends_likes['page_id'].isin(user_1_likes)]

# Оставляем только уникальные страницы
recommended_pages = recommended_pages['page_id'].drop_duplicates()

# Выводим рекомендуемые страницы
print("\nРекомендуемые страницы для пользователя 1:")
print(recommended_pages)
